In [ ]:
import pandas as pd
import matplotlib as plt

In [ ]:
%matplotlib inline

In [ ]:
recent_grads = pd.read_csv('recent-grads.csv')
print(recent_grads.iloc[0]

In [ ]:
recent_grads.head()

In [ ]:
recent_grads.tail()